# Import Packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import requests
import pandas as pd
# import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
import json
import time
import numpy as np
import h3
from folium.plugins import HeatMap
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
import time
import json

/Users/utkarshmaheshwari/opt/anaconda3/envs/whyhere/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
sys.path.append('src')

In [91]:
from fetch_data import *
from data_io import *
from data_prep import *
from scoring import *
from threshold_clustering import *
from dbscan_clustering import *
from visualization import *
from budget_filter import *
from recommendations import *

# Inputs

In [6]:
#### FRONTEND INPUTS ####

#the UI will instead give ranks, we need to convert to weights
# user_weights = {
#     'police_station': 6,
#     'grocery_store': 1,
#     'hospital': 5,
#     'marta_stop': 2,
#     'school': 0,
#     'restaurant': 3,
#     'park': 4
# }

user_weights = {
    "restaurant": 0.49,
    "grocery_store": 0.86,
    "school": 0.72,
    "hospital": 0.7,
    "marta_stop": 0.41,
    "police_station": 0.79,
    "park": 0.75,
    "crime_incident": 0.8
}



# radius in miles
user_radius_miles = 12/1.60934
user_has_vehicle = False
center = (33.749, -84.388)
budget = 1200


# convert to km
user_radius_km = user_radius_miles * 1.60934 

# Query Data

In [7]:
# all_pois = []
# all_pois = query_restaurant_data(ATLANTA_BBOX, all_pois)
# all_pois = query_park_data(ATLANTA_BBOX, all_pois)
# all_pois = query_hospital_and_clinic_data(ATLANTA_BBOX, all_pois)
# print(f"Total POIs fetched: {len(all_pois)}")

# Query Save & Load

In [8]:
name_of_the_file = "combine_datasets_v2"

In [9]:
# save_pois(all_pois, name_of_the_file)

In [10]:
df_pois = load_pois(name_of_the_file)

Loaded 170681
Summary by type:
type
school            102130
crime_incident     50202
marta_stop          9266
hospital            8013
park                 503
grocery_store        300
police_station       247
restaurant            20
Name: count, dtype: int64


# Basic EDA

In [11]:
df_pois.isnull().sum(axis=0)

type    0
name    0
lat     0
lon     0
dtype: int64

In [12]:
df_pois.groupby('type').agg({'lat':['min','max'] , 'lon':['min','max'], 'type':['count']})
#looks like the data has higher coverage than just Atlanta and Metro Atlanta

lat                    lon                type
                      min        max         min         max   count
type                                                                
crime_incident  32.151695  34.777687  -85.711189  -82.680045   50202
grocery_store   33.293267  34.261048  -84.888640  -83.886021     300
hospital       -14.290242  71.297725 -176.640263  145.724472    8013
marta_stop      33.432372  34.105822  -84.669803  -84.083455    9266
park            33.453098  33.976909  -84.552848  -84.153698     503
police_station  32.845390  34.557793  -85.287257  -83.596301     247
restaurant      33.761162  33.859863  -84.455464  -84.329030      20
school         -14.348924  71.300337 -176.640331  145.784430  102130

In [13]:
df_pois = keep_pois_within_bbox(df_pois, user_radius_km)
df_pois.head()

Filtered POIs from 170681 to 57626 within bbox


,type,name,lat,lon
0,police_station,DEKALB TECHNICAL COLLEGE POLICE,33.789938,-84.234406
1,police_station,DEKALB COUNTY SHERIFFS OFFICE / DEKALB COUNTY ...,33.775744,-84.244527
2,police_station,DEKALB COUNTY MARSHALS OFFICE,33.774070,-84.297214
3,police_station,GEORGIA BUREAU OF INVESTIGATION,33.692882,-84.272514
4,police_station,FULTON COUNTY MARSHALS OFFICE,33.750652,-84.391145


In [14]:
df_pois.groupby('type').agg({'lat':['min','max'] , 'lon':['min','max'], 'type':['count']})

lat                   lon              type
                      min        max        min        max  count
type                                                             
crime_incident  33.624855  33.878633 -84.543773 -84.252012  50129
grocery_store   33.620239  33.876639 -84.531024 -84.239643     62
hospital        33.680272  33.857813 -84.513144 -84.248184     25
marta_stop      33.619354  33.878604 -84.543062 -84.232170   6642
park            33.654428  33.876902 -84.538924 -84.284425    497
police_station  33.620332  33.849226 -84.540764 -84.234406     55
restaurant      33.761162  33.859863 -84.455464 -84.329030     20
school          33.621900  33.875800 -84.538900 -84.232900    196

In [15]:
df_pois['type'].unique()

array(['police_station', 'grocery_store', 'hospital', 'marta_stop',
       'school', 'restaurant', 'crime_incident', 'park'], dtype=object)

# Data Prep - Data Points

In [16]:
# either use boundaries or radius function to create hex grids
# hexagons = create_hex_grids_with_boundaries(df_pois)
hexagons = create_hex_grids_with_radius(df_pois, radius_km=user_radius_km, center = center, size_of_grid = 8)



Using circular boundary: center (33.7490, -84.3880), radius 12.0 km
Generated 890 hexagons (before filtering)
Filtered to 536 hexagons within 12.0 km of center


# Data Prep - Features

In [17]:
## define config for each POI type
# sameple config, the below dictionary represents the default settings for each POI type. For custom seetings, modify and pass it to calculate_accessibility_scores function

# poi_types_config = {
#     'restaurant': {'types': ['restaurant'], 'decay_rate': 1.5, 'max_distance_km': 10, 'invert': False},
#     'grocery_store': {'types': ['grocery_store'], 'decay_rate': 2, 'max_distance_km': 8, 'invert': False},
#     'school': {'types': ['school'], 'decay_rate': 1, 'max_distance_km': 15, 'invert': False},
#     'hospital': {'types': ['hospital'], 'decay_rate': 0.8, 'max_distance_km': 20, 'invert': False},
#     'marta_stop': {'types': ['marta_stop'], 'decay_rate': 0.5, 'max_distance_km': 5, 'invert': False},
#     'police_station': {'types': ['police_station'], 'decay_rate': 0.5, 'max_distance_km': 10, 'invert': False},
#     'park': {'types': ['park'], 'decay_rate': 1.0, 'max_distance_km': 5, 'invert': False},
#     'crime_incident': {'types': ['crime_incident'], 'decay_rate': 2.0, 'max_distance_km': 3, 'invert': True},
# }

In [18]:
# will take some time to run
# pass user_has_vehicle to the function later
df_hexagons = calculate_accessibility_scores(hexagons, df_pois, user_has_vehicle)

Calculating accessibility scores for each hexagon...
  Processing hexagon 0/536...
  Processing hexagon 25/536...
  Processing hexagon 50/536...
  Processing hexagon 75/536...
  Processing hexagon 100/536...
  Processing hexagon 125/536...
  Processing hexagon 150/536...
  Processing hexagon 175/536...
  Processing hexagon 200/536...
  Processing hexagon 225/536...
  Processing hexagon 250/536...
  Processing hexagon 275/536...
  Processing hexagon 300/536...
  Processing hexagon 325/536...
  Processing hexagon 350/536...
  Processing hexagon 375/536...
  Processing hexagon 400/536...
  Processing hexagon 425/536...
  Processing hexagon 450/536...
  Processing hexagon 475/536...
  Processing hexagon 500/536...
  Processing hexagon 525/536...

Calculated accessibility scores for 536 hexagons

Accessibility Score Statistics:
       restaurant_accessibility  grocery_store_accessibility  \
count                536.000000                   536.000000   
mean                   0.419494      

In [ ]:
df_budget = load_budget_data("Rent_atlanta")
df_budget_hex = convert_rent_data_to_h3(df_budget)
df_out = get_nearest_rent(df_budget_hex, hexagons, K=1)
df_hexagons = merge_budget_with_accessibility(df_hexagons, df_out)





Merging budget data with 536 with accessibility data with 536
Merge successful: all accessibility hexagons have budget data; total entries: 536


In [119]:
df_hexagons = smooth_scores_spatially(df_hexagons, neighbor_weight=0.3)


Applying spatial smoothing to 8 score columns...
  Smoothing hexagon 0/536...
  Smoothing hexagon 50/536...
  Smoothing hexagon 100/536...
  Smoothing hexagon 150/536...
  Smoothing hexagon 200/536...
  Smoothing hexagon 250/536...
  Smoothing hexagon 300/536...
  Smoothing hexagon 350/536...
  Smoothing hexagon 400/536...
  Smoothing hexagon 450/536...
  Smoothing hexagon 500/536...
Spatial smoothing complete


In [19]:
df_hexagons = filter_hexagons_by_budget(df_hexagons, max_budget=budget)


Before filtering: 536 hexagons
After filtering: 313 hexagons


In [84]:
# df_hexagons = apply_user_weights(df_hexagons, user_weights)


df_hexagons = apply_user_weights(df_hexagons, user_weights)



print(df_hexagons.head())

Applying user preferences: {'restaurant': 0.49, 'grocery_store': 0.86, 'school': 0.72, 'hospital': 0.7, 'marta_stop': 0.41, 'police_station': 0.79, 'park': 0.75, 'crime_incident': 0.85}
Normalized weights (exponential): {'restaurant': 0.14337991512507273, 'grocery_store': 0.11094486871232079, 'school': 0.1222507184143652, 'hospital': 0.12395727484738708, 'marta_stop': 0.1515551569413614, 'police_station': 0.11646067964969396, 'park': 0.11973483500961317, 'crime_incident': 0.11171655130018544}

User Match Score Statistics:
count    313.000000
mean       0.399157
std        0.149016
min        0.141104
25%        0.286525
50%        0.381022
75%        0.496429
max        0.794623
Name: user_match_score, dtype: float64
            hex_id        lat        lon  restaurant_accessibility  \
2  8844c1a865fffff  33.715965 -84.342377                  0.438628   
3  8844c1aa35fffff  33.705560 -84.390105                  0.413945   
4  8844c1a1abfffff  33.749898 -84.486790                  0.063

In [85]:
print(df_hexagons['user_match_score'].describe())

count    313.000000
mean       0.399157
std        0.149016
min        0.141104
25%        0.286525
50%        0.381022
75%        0.496429
max        0.794623
Name: user_match_score, dtype: float64


In [86]:
# Top 5 hexagons
top_5 = df_hexagons.nlargest(5, 'user_match_score')
print(top_5[['hex_id', 'user_match_score', 'restaurant_accessibility', 'grocery_store_accessibility']])

# Bottom 5 hexagons
bottom_5 = df_hexagons.nsmallest(5, 'user_match_score')
print(bottom_5[['hex_id', 'user_match_score', 'restaurant_accessibility', 'grocery_store_accessibility']])

              hex_id  user_match_score  restaurant_accessibility  \
481  8844c1a8e9fffff          0.794623                  2.384242   
404  8844c1a817fffff          0.774579                  1.783589   
486  8844c1a8c7fffff          0.739854                  2.127873   
238  8844c1a8c5fffff          0.738196                  1.913084   
535  8844c1a9cbfffff          0.733761                  2.070169   

     grocery_store_accessibility  
481                     1.066123  
404                     0.803686  
486                     0.957885  
238                     0.834557  
535                     1.227367  
              hex_id  user_match_score  restaurant_accessibility  \
86   8844c1ab61fffff          0.141104                  0.000000   
435  8844c1ab45fffff          0.145877                  0.000000   
192  8844c1ab67fffff          0.151111                  0.002145   
115  8844c1ab6bfffff          0.155334                  0.001647   
51   8844c1ab49fffff          0.155364   

# Percentile Based Clustering

In [23]:
# df_threshold = cluster_based_on_score(df_hexagons)
# df_threshold.head()


df_classified = cluster_based_on_score(
    df_hexagons,
    n_tiers=10
)

Classifying into 10 tiers:
  Tier 0 threshold (top 90.0%): 0.450
  Tier 1 threshold (top 80.0%): 0.355
  Tier 2 threshold (top 70.0%): 0.285
  Tier 3 threshold (top 60.0%): 0.248
  Tier 4 threshold (top 50.0%): 0.214
  Tier 5 threshold (top 40.0%): 0.189
  Tier 6 threshold (top 30.0%): 0.155
  Tier 7 threshold (top 20.0%): 0.111
  Tier 8 threshold (top 10.0%): 0.064

Suitability Distribution:
suitability_label
Less Suitable    31
Most Suitable    32
Okay             31
Name: count, dtype: int64

SUITABILITY TIER CHARACTERISTICS

Most Suitable (32 hexagons):
  Match Score Range: 0.450 - 0.830
  Avg restaurant: 1.408
  Avg grocery_store: 0.732
  Avg school: 18.166

Okay (31 hexagons):
  Match Score Range: 0.357 - 0.450
  Avg restaurant: 0.617
  Avg grocery_store: 0.397
  Avg school: 18.234

Less Suitable (31 hexagons):
  Match Score Range: 0.286 - 0.353
  Avg restaurant: 0.420
  Avg grocery_store: 0.318
  Avg school: 16.801


In [28]:
#not required, will be handles by UI

threshold_map_name = "data/output_data/atlanta_threshold_map_v2.html"
map_threshold = create_suitability_map(df_classified, user_weights)
map_threshold.save(threshold_map_name)

Adding hexagons to map...
  Added 50/313 hexagons...
  Added 100/313 hexagons...
  Added 200/313 hexagons...
  Added 250/313 hexagons...
  Added 350/313 hexagons...
  Added 400/313 hexagons...
  Added 450/313 hexagons...
  Added 500/313 hexagons...


In [29]:
save_csv(df_classified, "combined_data_outputs")

Saved CSV: data/output_data/combined_data_outputs.csv


In [30]:
df_classified_json = convert_json(df_classified)

In [31]:
save_json(df_classified_json, "combined_data_outputs_json")

In [34]:
df_classified.head().columns.tolist()

['hex_id',
 'lat',
 'lon',
 'restaurant_accessibility',
 'grocery_store_accessibility',
 'school_accessibility',
 'hospital_accessibility',
 'marta_stop_accessibility',
 'police_station_accessibility',
 'park_accessibility',
 'crime_incident_accessibility',
 'avg_rent',
 'police_station_norm',
 'grocery_store_norm',
 'hospital_norm',
 'marta_stop_norm',
 'restaurant_norm',
 'park_norm',
 'user_match_score',
 'suitability',
 'suitability_label']

# Experiment 2: DBSCAN Clustering

In [14]:
df_dbscan = dbscan_score_clustering(df_hexagons, eps=0.1, min_samples=3)


DBSCAN CLUSTERING (Score-Based)
Parameters: eps=0.1, min_samples=3
Score range after scaling: [0.000, 1.000]

Results:
  Clusters found: 1
  Noise points: 0
  Cluster 0: 536 hexagons, avg score = 0.161


In [15]:
cluster_colors = get_cluster_colors(df_dbscan)

In [16]:
dbscan_map_name = "data/output_data/atlanta_dbscan_map.html"
map_dbscan = create_dbscan_map(df_dbscan, user_weights, cluster_colors=cluster_colors, use_heatmap=True, heatmap_radius=15)
map_dbscan.save(dbscan_map_name)

Adding hexagons to map...
  Added 0/536 hexagons...
  Added 50/536 hexagons...
  Added 100/536 hexagons...
  Added 150/536 hexagons...
  Added 200/536 hexagons...
  Added 250/536 hexagons...
  Added 300/536 hexagons...
  Added 350/536 hexagons...
  Added 400/536 hexagons...
  Added 450/536 hexagons...
  Added 500/536 hexagons...
Adding heatmap overlay for smooth visualization...


# Experiment 3 - Sptial DBSCAN

In [17]:
df_dbscan_spatial = dbscan_spatial_clustering(
    df_hexagons,
    eps=0.2,
    min_samples=3,
    spatial_weight=0.4
)


DBSCAN CLUSTERING (Spatially-Aware)
Parameters: eps=0.2, min_samples=3, spatial_weight=0.4

Results:
  Clusters found: 6
  Noise points: 1
  Noise/Uncertain: 1 hexagons
  Region 0: 502 hexagons, avg score = 0.127, extent = 33.1 km
  Region 1: 5 hexagons, avg score = 0.783, extent = 3.7 km
  Region 2: 15 hexagons, avg score = 0.590, extent = 7.3 km
  Region 3: 5 hexagons, avg score = 0.699, extent = 4.9 km
  Region 4: 5 hexagons, avg score = 0.899, extent = 2.6 km
  Region 5: 3 hexagons, avg score = 0.521, extent = 3.4 km


In [18]:
cluster_colors_spatial = get_cluster_colors(df_dbscan_spatial)

In [19]:
map_dbscan_spatial = create_dbscan_map(
    df_dbscan_spatial,
    user_weights,
    cluster_colors=cluster_colors_spatial,
    use_heatmap=True,
    heatmap_radius=15
)

spatial_dbscan_map_name = "data/output_data/atlanta_spatial_dbscan_map.html"
map_dbscan_spatial.save(spatial_dbscan_map_name)

Adding hexagons to map...
  Added 0/536 hexagons...
  Added 50/536 hexagons...
  Added 100/536 hexagons...
  Added 150/536 hexagons...
  Added 200/536 hexagons...
  Added 250/536 hexagons...
  Added 300/536 hexagons...
  Added 350/536 hexagons...
  Added 400/536 hexagons...
  Added 450/536 hexagons...
  Added 500/536 hexagons...
Adding heatmap overlay for smooth visualization...


# Collaborative Filtering

## Create Bsaeline Data

In [93]:
user_profiles = [
    {
        "name": "young_family_with_toddler",
        "restaurant": 0.6,
        "grocery_store": 0.9,
        "school": 0.95,
        "hospital": 0.8,
        "marta_stop": 0,
        "police_station": 0.7,
        "park": 0.95,
        "crime_incident": 0.85
    },
    {
        "name": "university_student",
        "restaurant": 0.8,
        "grocery_store": 0.6,
        "school": 0,
        "hospital": 0.3,
        "marta_stop": 0.95,
        "police_station": 0.5,
        "park": 0.4,
        "crime_incident": 0.6
    },
    {
        "name": "young_professional_no_car",
        "restaurant": 0.9,
        "grocery_store": 0.7,
        "school": 0.1,
        "hospital": 0.5,
        "marta_stop": 0.95,
        "police_station": 0.6,
        "park": 0.5,
        "crime_incident": 0.7
    },
    {
        "name": "retiree_couple",
        "restaurant": 0.5,
        "grocery_store": 0.8,
        "school": 0,
        "hospital": 0.95,
        "marta_stop": 0,
        "police_station": 0.8,
        "park": 0.9,
        "crime_incident": 0.9
    },
    {
        "name": "remote_worker_homebody",
        "restaurant": 0.4,
        "grocery_store": 0.9,
        "school": 0,
        "hospital": 0.6,
        "marta_stop": 0,
        "police_station": 0.7,
        "park": 0,
        "crime_incident": 0.85
    },
    {
        "name": "foodie_socialite",
        "restaurant": 0.95,
        "grocery_store": 0,
        "school": 0,
        "hospital": 0.4,
        "marta_stop": 0,
        "police_station": 0.5,
        "park": 0,
        "crime_incident": 0.5
    },
    {
        "name": "health_conscious_athlete",
        "restaurant": 0.6,
        "grocery_store": 0.8,
        "school": 0.1,
        "hospital": 0.7,
        "marta_stop": 0.4,
        "police_station": 0.6,
        "park": 0.95,
        "crime_incident": 0.7
    },
    {
        "name": "family_with_teenagers",
        "restaurant": 0.7,
        "grocery_store": 0.85,
        "school": 0.9,
        "hospital": 0.7,
        "marta_stop": 0.6,
        "police_station": 0.8,
        "park": 0.7,
        "crime_incident": 0.85
    },
    {
        "name": "elderly_living_alone",
        "restaurant": 0.3,
        "grocery_store": 0.95,
        "school": 0.1,
        "hospital": 0.95,
        "marta_stop": 0.5,
        "police_station": 0.9,
        "park": 0.6,
        "crime_incident": 0.95
    },
    {
        "name": "graduate_student_researcher",
        "restaurant": 0.6,
        "grocery_store": 0.7,
        "school": 0,
        "hospital": 0.4,
        "marta_stop": 0.9,
        "police_station": 0.5,
        "park": 0.5,
        "crime_incident": 0.65
    },
    {
        "name": "young_couple_no_kids",
        "restaurant": 0.85,
        "grocery_store": 0.7,
        "school": 0.1,
        "hospital": 0.5,
        "marta_stop": 0.8,
        "police_station": 0.6,
        "park": 0.7,
        "crime_incident": 0.7
    },
    {
        "name": "single_parent_two_kids",
        "restaurant": 0.5,
        "grocery_store": 0.9,
        "school": 0.95,
        "hospital": 0.85,
        "marta_stop": 0.7,
        "police_station": 0.85,
        "park": 0.8,
        "crime_incident": 0.9
    },
    {
        "name": "safety_focused_conservative",
        "restaurant": 0.4,
        "grocery_store": 0.8,
        "school": 0.6,
        "hospital": 0.8,
        "marta_stop": 0.3,
        "police_station": 0.95,
        "park": 0.6,
        "crime_incident": 0.95
    },
    {
        "name": "eco_warrior_cyclist",
        "restaurant": 0.7,
        "grocery_store": 0.8,
        "school": 0.3,
        "hospital": 0.5,
        "marta_stop": 0.9,
        "police_station": 0.5,
        "park": 0.95,
        "crime_incident": 0.6
    },
    {
        "name": "busy_executive",
        "restaurant": 0.8,
        "grocery_store": 0.6,
        "school": 0.2,
        "hospital": 0.7,
        "marta_stop": 0.85,
        "police_station": 0.6,
        "park": 0.4,
        "crime_incident": 0.7
    },
    {
        "name": "artist_creative_type",
        "restaurant": 0.75,
        "grocery_store": 0.6,
        "school": 0.2,
        "hospital": 0.4,
        "marta_stop": 0.8,
        "police_station": 0.4,
        "park": 0.8,
        "crime_incident": 0.5
    },
    {
        "name": "medical_professional",
        "restaurant": 0.6,
        "grocery_store": 0.7,
        "school": 0.5,
        "hospital": 0.95,
        "marta_stop": 0.6,
        "police_station": 0.7,
        "park": 0.5,
        "crime_incident": 0.75
    },
    {
        "name": "teacher_educator",
        "restaurant": 0.5,
        "grocery_store": 0.8,
        "school": 0.9,
        "hospital": 0.6,
        "marta_stop": 0.7,
        "police_station": 0.7,
        "park": 0.75,
        "crime_incident": 0.8
    },
    {
        "name": "budget_conscious_minimalist",
        "restaurant": 0.3,
        "grocery_store": 0.95,
        "school": 0.2,
        "hospital": 0.6,
        "marta_stop": 0.85,
        "police_station": 0.6,
        "park": 0.5,
        "crime_incident": 0.7
    },
    {
        "name": "suburban_transplant",
        "restaurant": 0.5,
        "grocery_store": 0.85,
        "school": 0.7,
        "hospital": 0.7,
        "marta_stop": 0.4,
        "police_station": 0.8,
        "park": 0.8,
        "crime_incident": 0.85
    }
]
df_user_profiles = pd.DataFrame(user_profiles)

In [96]:
import random

# counter = 1
liked_places_list = []
for idx, row in df_user_profiles.iterrows():

    row = row.drop('name').to_dict()
    df_hexagons_fake = calculate_accessibility_scores(hexagons, df_pois, user_has_vehicle) 
    df_hexagons_fake = smooth_scores_spatially(df_hexagons_fake, neighbor_weight=0.3)
    df_hexagons_fake = apply_user_weights(df_hexagons_fake, row)
    df_classified_fake = cluster_based_on_score(df_hexagons_fake,n_tiers=10)

    tier_0_hexagons = df_classified_fake[df_classified_fake['suitability'] == 0]['hex_id'].tolist()
    tier_1_hexagons = df_classified_fake[df_classified_fake['suitability'] == 1]['hex_id'].tolist()
    liked_from_tier_0 = random.sample(tier_0_hexagons, min(3, len(tier_0_hexagons)))
    liked_from_tier_1 = random.sample(tier_1_hexagons, min(2, len(tier_1_hexagons)))
    liked_places = liked_from_tier_0 + liked_from_tier_1

    liked_places_list.append(liked_places)




>> Using cached accessibility scores

Applying spatial smoothing to 8 score columns...
  Smoothing hexagon 0/536...
  Smoothing hexagon 50/536...
  Smoothing hexagon 100/536...
  Smoothing hexagon 150/536...
  Smoothing hexagon 200/536...
  Smoothing hexagon 250/536...
  Smoothing hexagon 300/536...
  Smoothing hexagon 350/536...
  Smoothing hexagon 400/536...
  Smoothing hexagon 450/536...
  Smoothing hexagon 500/536...
Spatial smoothing complete
Applying user preferences: {'restaurant': 0.6, 'grocery_store': 0.9, 'school': 0.95, 'hospital': 0.8, 'marta_stop': 0.0, 'police_station': 0.7, 'park': 0.95, 'crime_incident': 0.85}
Normalized weights (exponential): {'restaurant': 0.16595029899213465, 'grocery_store': 0.1347935280323951, 'school': 0.13020196563044004, 'hospital': 0.144468126266763, 'police_station': 0.15483710391505903, 'park': 0.13020196563044004, 'crime_incident': 0.13954701153276813}

User Match Score Statistics:
count    536.000000
mean       0.400060
std        0.114392


In [97]:
df_user_profiles['liked_places'] = liked_places_list
df_user_profiles

,name,restaurant,grocery_store,school,hospital,marta_stop,police_station,park,crime_incident,liked_places
0,young_family_with_toddler,0.60,0.90,0.95,0.80,0.00,0.70,0.95,0.85,"[8844c1a8b7fffff, 8844c1a83bfffff, 8844c1a9d1f..."
1,university_student,0.80,0.60,0.00,0.30,0.95,0.50,0.40,0.60,"[8844c1a8a3fffff, 8844c1a8c7fffff, 8844c1a8e3f..."
2,young_professional_no_car,0.90,0.70,0.10,0.50,0.95,0.60,0.50,0.70,"[8844c1a8e3fffff, 8844c1a8a1fffff, 8844c1a99df..."
3,retiree_couple,0.50,0.80,0.00,0.95,0.00,0.80,0.90,0.90,"[8844c1a837fffff, 8844c1a8bbfffff, 8844c1a833f..."
4,remote_worker_homebody,0.40,0.90,0.00,0.60,0.00,0.70,0.00,0.85,"[8844c1a8b7fffff, 8844c13245fffff, 8844c1a8e3f..."
5,foodie_socialite,0.95,0.00,0.00,0.40,0.00,0.50,0.00,0.50,"[8844c1a9dbfffff, 8844c1a9ddfffff, 8844c1a833f..."
6,health_conscious_athlete,0.60,0.80,0.10,0.70,0.40,0.60,0.95,0.70,"[8844c1a8a3fffff, 8844c1a9d5fffff, 8844c13249f..."
7,family_with_teenagers,0.70,0.85,0.90,0.70,0.60,0.80,0.70,0.85,"[8844c1a839fffff, 8844c1a803fffff, 8844c1a8edf..."
8,elderly_living_alone,0.30,0.95,0.10,0.95,0.50,0.90,0.60,0.95,"[8844c1a9ddfffff, 8844c1aaa5fffff, 8844c1a993f..."
9,graduate_student_researcher,0.60,0.70,0.00,0.40,0.90,0.50,0.50,0.65,"[8844c1a813fffff, 8844c1a991fffff, 8844c1a83bf..."


## Find recommendations

In [90]:
from sklearn.neighbors import NearestNeighbors

feature_cols = [col for col in df_user_profiles.columns if (col != 'name' and col != 'liked_places')]
samples = df_user_profiles[feature_cols].values


neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(samples)

fake_user_array = [user_weights[col] for col in feature_cols]
distances, indices = neigh.kneighbors([fake_user_array])

similarity = 1 - distances[0][0]

print("Users like you liked these places:")
print(df_user_profiles.iloc[indices[0][0], -1])
print(f"matching score: {similarity*100: .0f}%")


Users like you liked these places:
['8844c1a9dbfffff', '8844c1a8a9fffff', '8844c1a819fffff', '8844c1a8b1fffff', '8844c1a835fffff']
matching score:  92%


In [107]:
a, b = generate_recommendations(user_weights)

In [108]:
a

['8844c1a839fffff',
 '8844c1a83bfffff',
 '8844c1a819fffff',
 '8844c1a889fffff',
 '8844c1a801fffff']

In [109]:
b

np.float64(0.9238422689413609)